In [1]:
# -*- encoding: utf-8 -*-
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import re

In [2]:
data_dir = './Batik300'
value_folder = os.listdir(data_dir)
total_value_folder = len(value_folder)
#print (value_folder[:5])


In [3]:
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    return [ atof(c) for c in re.split(r'[+-_]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

In [4]:
value_folder.sort(key=natural_keys)
#print(value_folder[:20])

In [5]:
X_train, Y_train = [], []
X_test, Y_test = [], []


In [6]:
# Iterate through the list of subfolders (subjects)
# Idx is the subject ID
idx = 0
for a in range (0,300,6):
    for train in range(5):
        img = load_img(os.path.join(data_dir,value_folder[a+train]))
        img = img_to_array(img).astype('float32')/255
        X_train.append(img)
        Y_train.append(idx)
    for test in range(5,6):
        img = load_img(os.path.join(data_dir,value_folder[a+test]))
        img = img_to_array(img).astype('float32')/255
        X_test.append(img)
        Y_test.append(idx)
    idx=idx+1    

In [7]:

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)



In [8]:
print (len(X_train),len(X_test))

250 50


In [9]:
num_classes = len(np.unique(Y_train))


In [10]:
def euclidean_distance(vectors):
    vector1, vector2 = vectors
    sum_square = tf.reduce_sum(tf.square(vector1 - vector2), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, tf.keras.backend.epsilon())) 

def contrastive_loss(Y_true,D):
    margin = 1
    
    loss= tf.reduce_mean(Y_true*tf.math.square(D)+(1 - Y_true)*tf.math.maximum((margin-D),0))
    return loss


In [11]:
def SiameseNetwork(ukuran):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=ukuran, kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05,seed=100)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05,seed=100)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05,seed=100)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='sigmoid')])
    
    return model
        

In [12]:
ukuran=(128,128,3)
tf.random.set_seed(100)
shared_network = SiameseNetwork(ukuran)
shared_network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0

In [13]:
input_top = tf.keras.Input(shape=ukuran)
input_bottom = tf.keras.Input(shape=ukuran)
output_top = shared_network(input_top)
output_bottom = shared_network(input_bottom)

distance = Lambda(euclidean_distance, output_shape=(1,))([output_top,output_bottom])  
model = tf.keras.Model(inputs=[input_top, input_bottom], outputs=distance)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128)          1681408     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           sequential[1][0]             

In [14]:
def create_pairs(X,Y, num_classes):
    pairs, labels = [], []
    # index of images in X and Y for each class
    class_idx = [np.where(Y==i)[0] for i in range(num_classes)]
    # The minimum number of images across all classes
    min_images = min(len(class_idx[i]) for i in range(num_classes)) - 1
    for c in range(num_classes):
        for n in range(min_images):
            # create positive pair
            img1 = X[class_idx[c][n]]
            img2 = X[class_idx[c][n+1]]
            pairs.append((img1, img2))
            labels.append(1)
            # create negative pair
            # list of classes that are different from the current class
            neg_list = list(range(num_classes))
            neg_list.remove(c)
            # select a random class from the negative list.
            # This class will be used to form the negative pair.
            neg_c = random.sample(neg_list,1)[0]
            img1 = X[class_idx[c][n]]
            img2 = X[class_idx[neg_c][n]]
            pairs.append((img1,img2))
            labels.append(0)
    return np.array(pairs), np.array(labels)


num_classes = len(np.unique(Y_train))
training_pairs, training_labels = create_pairs(X_train, Y_train,len(np.unique(Y_train)))
test_pairs, test_labels = create_pairs(X_test, Y_test,len(np.unique(Y_test)))

In [15]:
model.compile(loss=contrastive_loss, optimizer='adam')

In [16]:
model.fit([training_pairs[:, 0], training_pairs[:, 1]], training_labels,batch_size=64, epochs=20)

Epoch 1/20
7/7 [==============================] - 20s 3s/step - loss: 0.2212
Epoch 2/20
7/7 [==============================] - 21s 3s/step - loss: 0.0895
Epoch 3/20
7/7 [==============================] - 21s 3s/step - loss: 0.0736
Epoch 4/20
7/7 [==============================] - 22s 3s/step - loss: 0.0555
Epoch 5/20
7/7 [==============================] - 19s 3s/step - loss: 0.0494
Epoch 6/20
7/7 [==============================] - 20s 3s/step - loss: 0.0339
Epoch 7/20
7/7 [==============================] - 20s 3s/step - loss: 0.0318
Epoch 8/20
7/7 [==============================] - 20s 3s/step - loss: 0.0260
Epoch 9/20
7/7 [==============================] - 19s 3s/step - loss: 0.0271
Epoch 10/20
7/7 [==============================] - 18s 3s/step - loss: 0.0298
Epoch 11/20
7/7 [==============================] - 18s 3s/step - loss: 0.0203
Epoch 12/20
7/7 [==============================] - 19s 3s/step - loss: 0.0182
Epoch 13/20
7/7 [==============================] - 20s 3s/step - loss: 0.

In [17]:
data_for_matching = []

for a in range(0,300,6):
    img = load_img(os.path.join(data_dir,value_folder[a]))
    img = img_to_array(img).astype('float32')/255
    data_for_matching.append(img)
    
data_for_matching = np.array(data_for_matching)

print (len(data_for_matching))

50


In [18]:
kamus = dict({0: 'm-1', 1: 'm-2',
              2: 'm-3', 3: 'm-4',
              4: 'm-5', 5: 'm-6',
              6: 'm-7', 7: 'm-8',
              8: 'm-9', 9: 'm-10',
              10: 'm-11',11: 'm-12',
              12: 'm-13',13: 'm-14',
              14: 'm-15',15: 'm-16',
              16: 'm-17',17: 'm-18',
              18: 'm-19',19: 'm-20',
              
              20: 'm-21', 21: 'm-22',
              22: 'm-23', 23: 'm-24',
              24: 'm-25', 25: 'm-26',
              26: 'm-27', 27: 'm-28',
              28: 'm-29', 29: 'm-30',
              30: 'm-31', 31: 'm-32',
              32: 'm-33', 33: 'm-34',
              34: 'm-35', 35: 'm-36',
              36: 'm-37', 37: 'm-38',
              38: 'm-39', 39: 'm-40',
               
              40: 'm-41', 41: 'm-42',
              42: 'm-43', 43: 'm-44',
              44: 'm-45', 45: 'm-46',
              46: 'm-47', 47: 'm-48',
              48: 'm-49', 49: 'm-50'})


In [20]:
hitung_true=0
hitung_false=0
nama_kelas=1
for a in range(0,300,6):
    hasil=[]
    img_test = load_img(os.path.join(data_dir,value_folder[a+5]))
    img_test = img_to_array(img_test).astype('float32')/255
    img_test = np.array(img_test)
    img_test = np.expand_dims(img_test, axis=0)
    for c in range(len(data_for_matching)):
        image_data = data_for_matching[c]
        image_data = np.expand_dims(image_data, axis=0)
        dissimilarity = model.predict([image_data, img_test])[0][0]
        hasil.append(dissimilarity)
        nilai_terendah = np.argmin(hasil)
    print ("File : ",value_folder[a+5],"model : ",kamus[nama_kelas-1],", predicted result : ",kamus[nilai_terendah])
    if nama_kelas-1 == nilai_terendah:
        hitung_true = hitung_true+1
    else:
        hitung_false=hitung_false+1
    nama_kelas=nama_kelas+1

File :  B1_6.jpg model :  m-1 , predicted result :  m-1
File :  B2_6.jpg model :  m-2 , predicted result :  m-2
File :  B3_6.jpg model :  m-3 , predicted result :  m-3
File :  B4_6.jpg model :  m-4 , predicted result :  m-4
File :  B5_6.jpg model :  m-5 , predicted result :  m-5
File :  B6_6.jpg model :  m-6 , predicted result :  m-6
File :  B7_6.jpg model :  m-7 , predicted result :  m-7
File :  B8_6.jpg model :  m-8 , predicted result :  m-27
File :  B9_6.jpg model :  m-9 , predicted result :  m-9
File :  B10_6.jpg model :  m-10 , predicted result :  m-49
File :  B11_6.jpg model :  m-11 , predicted result :  m-11
File :  B12_6.jpg model :  m-12 , predicted result :  m-12
File :  B13_6.jpg model :  m-13 , predicted result :  m-13
File :  B14_6.jpg model :  m-14 , predicted result :  m-14
File :  B15_6.jpg model :  m-15 , predicted result :  m-15
File :  B16_6.jpg model :  m-16 , predicted result :  m-16
File :  B17_6.jpg model :  m-17 , predicted result :  m-17
File :  B18_6.jpg model

In [21]:
print ("True predicted : ",hitung_true," False predicted: ",hitung_false)


True predicted :  43  False predicted:  7


In [22]:
model.save('recognize_batik-siamese-final-v3.h5')